In [43]:
import cv2
import pandas as pd

def create_annotated_video(df: pd.DataFrame, 
                           class_label: float,
                           input_video_path: str,
                           output_video_path: str):

    
    # Initialize video capture from the input file
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Could not open video file: {input_video_path}")
        return
    
    # Read video info
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    
    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # 'mp4v' for MP4 output
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    frame_idx = 0  # We'll match this with df['image_idx']
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # We've reached the end of the input video
        
        # Filter the DataFrame to bounding boxes for this frame and the desired class
        frame_df = df[(df['image_idx'] == frame_idx) & (df['label'] == class_label)&(df['confidence'] > 0.3)]
        
        # Draw the bounding boxes
        for _, row in frame_df.iterrows():                                         
            conf    = row['confidence'] 
            x_center = row['x']   # normalized center
            y_center = row['y']   # normalized center
            w_norm   = row['w']   # normalized width
            h_norm   = row['h']   # normalized height                       

            # Convert normalized -> pixels
            x1 = int( (x_center - w_norm/2) )
            y1 = int( (y_center - h_norm/2)  )
            x2 = int( (x_center + w_norm/2) )
            y2 = int( (y_center + h_norm/2)  )
            
            # Draw a red rectangle
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 4)
            
            # Annotation text (bf + confidence)
            text = f"bf {conf:.2f}"
            cv2.putText(frame, text, (x1, max(0, y1 - 5)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1,  # font scale
                        (0, 0, 255), 
                        2,    # line thickness
                        cv2.LINE_AA)
        
        # Write the annotated frame to the output video
        out.write(frame)
        
        frame_idx += 1
    
    cap.release()
    out.release()
    print(f"Annotated video saved as: {output_video_path}")


In [44]:
import pandas as pd

# First load the CSV file into a DataFrame
df = pd.read_csv("/home/bohbot/Evyatar/exp/Joni_showoff/predict/results.csv")

# Then call your function, passing the actual DataFrame (df)
create_annotated_video(
    df,                       # <-- pass the DataFrame, not the CSV path
    1,                        # class_label
    "/home/bohbot/Evyatar/exp/Joni_showoff/full movies/bf.mp4", 
    "/home/bohbot/Evyatar/exp/Joni_showoff/predict/joni.mp4"
)


Annotated video saved as: /home/bohbot/Evyatar/exp/Joni_showoff/predict/joni.mp4
